In [1]:
# PROXY
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
    
os.environ['HF_HOME'] = '/autodl-tmp/hf'

In [2]:
from datasets import load_dataset
datas = load_dataset("bigbio/biored")

/root/miniconda3/lib/python3.8/site-packages/datasets/load.py:1461: FutureWarning: The repository for bigbio/biored contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/biored
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
from text2knowledge.strategy1 import extract_concepts, graph_prompt

import json

In [4]:
model_name = 'llama3:latest'

In [15]:
import os, sys
import contextlib

i = 4
while i < len(datas['test']):
    data = datas['test'][i]
    pmid = data['pmid']
    passages = data['passages']
    entities = data['entities']
    abstract = passages[1]['text'][0]
    abstract_offset = passages[1]['offsets'][0]
    metadata = {
        'pmid': pmid,
        'type': 'abstract',
    }

    #with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    entities_extracted = extract_concepts(abstract, model=model_name, metadata=metadata)
    
    if entities_extracted:
        disease_entities_extracted = [e for e in entities_extracted if e['category'] == 'Disease']
        # print(disease_entities_extracted)
        entities_str = json.dumps(entities_extracted, indent=4)
        # print(entities_str)
    else:
        print("No entities found.")
    break
    # i += 1
    # if i >= 2:
    #     break

NameError: name 'extract_concepts' is not defined

In [10]:
entities

[{'text': ['PKC-alpha'],
  'offsets': [[11, 20]],
  'concept_id': '18750',
  'semantic_type_id': 'GeneOrGeneProduct'},
 {'text': ['lithium'],
  'offsets': [[32, 39]],
  'concept_id': 'D008094',
  'semantic_type_id': 'ChemicalEntity'},
 {'text': ['nephrogenic diabetes insipidus'],
  'offsets': [[48, 78]],
  'concept_id': 'D018500',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['Lithium'],
  'offsets': [[80, 87]],
  'concept_id': 'D008094',
  'semantic_type_id': 'ChemicalEntity'},
 {'text': ['nephrogenic diabetes insipidus'],
  'offsets': [[125, 155]],
  'concept_id': 'D018500',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['NDI'],
  'offsets': [[157, 160]],
  'concept_id': 'D018500',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['patients'],
  'offsets': [[173, 181]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['lithium'],
  'offsets': [[244, 251]],
  'concept_id': 'D008094',
  'semantic_type_id': 'Chem

In [15]:
entities_extracted

In [16]:
disease_entities_extracted = [e for e in entities_extracted if e['category'] == 'Disease']
disease_entities_extracted

[{'entity': 'nephrogenic diabetes insipidus (NDI)',
  'confidence': '5',
  'category': 'Disease',
  'pmid': '25006961',
  'type': 'abstract'}]

In [8]:
import os, sys
import contextlib

model_name = 'llama3:latest'

i = 4
while i < len(datas['test']):
    data = datas['test'][i]
    pmid = data['pmid']
    passages = data['passages']
    relations = data['relations']
    abstract = passages[1]['text'][0]
    abstract_offset = passages[1]['offsets'][0]
    metadata = {
        'pmid': pmid,
        'type': 'abstract',
    }

    with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
        relations_extracted = graph_prompt(abstract, model=model_name, metadata=metadata)
    
    if relations_extracted:
        # disease_entities_extracted = [e for e in entities_extracted if e['category'] == 'Disease']
        # print(disease_entities_extracted)
        entities_str = json.dumps(relations_extracted, indent=4)
        print(entities_str)
    else:
        print("No entities found.")
    # break
    i += 1
    if i >= 6:
        break

[
    {
        "source_name": "Deoxyguanosine kinase",
        "source_type": "MolecularFunction",
        "target_name": "Deficiency",
        "target_type": "Condition",
        "relation_type": "BioMedGPS::AssociatedWith::Gene:Condition",
        "pmid": "19394258",
        "type": "abstract"
    },
    {
        "source_name": "Mitochondrial DNA depletion",
        "source_type": "Condition",
        "target_name": "Hepatocerebral phenotype",
        "target_type": "Anatomy",
        "relation_type": "BioMedGPS::AssociatedWith::Condition:Anatomy",
        "pmid": "19394258",
        "type": "abstract"
    },
    {
        "source_name": "DGUOK gene",
        "source_type": "Gene",
        "target_name": "Splice site mutation",
        "target_type": "Concept",
        "relation_type": "BioMedGPS::Modulator::Gene:Concept",
        "pmid": "19394258",
        "type": "abstract"
    },
    {
        "source_name": "Clinical features",
        "source_type": "Anatomy",
        "target

In [37]:
[(r, i) for i,d in enumerate(datas['test']) for r in d['relations'] if r['type'] == 'Bind']

[({'novel': 'No', 'type': 'Bind', 'concept_1': 'D013134', 'concept_2': '1813'},
  11),
 ({'novel': 'No', 'type': 'Bind', 'concept_1': 'D013134', 'concept_2': '1812'},
  11),
 ({'novel': 'No', 'type': 'Bind', 'concept_1': 'C534628', 'concept_2': '1813'},
  11),
 ({'novel': 'No', 'type': 'Bind', 'concept_1': 'C534628', 'concept_2': '1812'},
  11),
 ({'novel': 'No',
   'type': 'Bind',
   'concept_1': '822075',
   'concept_2': '836269'},
  14),
 ({'novel': 'No', 'type': 'Bind', 'concept_1': 'D002118', 'concept_2': '2979'},
  16),
 ({'novel': 'Novel', 'type': 'Bind', 'concept_1': '84515', 'concept_2': '595'},
  68),
 ({'novel': 'No', 'type': 'Bind', 'concept_1': '7421', 'concept_2': 'D014807'},
  71),
 ({'novel': 'No', 'type': 'Bind', 'concept_1': '2022', 'concept_2': '7040'},
  76)]

In [38]:
datas['test'][11]

{'pmid': '8829135',
 'passages': [{'type': 'title',
   'text': ['Nefiracetam (DM-9384) reverses apomorphine-induced amnesia of a passive avoidance response: delayed emergence of the memory retention effects.'],
   'offsets': [[0, 142]]},
  {'type': 'abstract',
   'text': ['Nefiracetam is a novel pyrrolidone derivative which attenuates scopolamine-induced learning and post-training consolidation deficits. Given that apomorphine inhibits passive avoidance retention when given during training or in a defined 10-12h post-training period, we evaluated the ability of nefiracetam to attenuate amnesia induced by dopaminergic agonism. A step-down passive avoidance paradigm was employed and nefiracetam (3 mg/kg) and apomorphine (0.5 mg/kg) were given alone or in combination during training and at the 10-12h post-training period of consolidation. Co-administration of nefiracetam and apomorphine during training or 10h thereafter produced no significant anti-amnesic effect. However, administration 

In [7]:
set([tri['type'] for d in datas['test'] for tri in d['relations']])

{'Association',
 'Bind',
 'Comparison',
 'Conversion',
 'Cotreatment',
 'Drug_Interaction',
 'Negative_Correlation',
 'Positive_Correlation'}

In [11]:
[tri for d in datas['test'] for tri in d['relations'] if tri['type'] == 'Comparison']

[{'novel': 'Novel',
  'type': 'Comparison',
  'concept_1': 'D019259',
  'concept_2': 'D000068698'},
 {'novel': 'No',
  'type': 'Comparison',
  'concept_1': 'D010656',
  'concept_2': 'D004809'},
 {'novel': 'Novel',
  'type': 'Comparison',
  'concept_1': 'D007479',
  'concept_2': 'C044834'},
 {'novel': 'Novel',
  'type': 'Comparison',
  'concept_1': 'C108761',
  'concept_2': 'D014635'},
 {'novel': 'Novel',
  'type': 'Comparison',
  'concept_1': 'D020927',
  'concept_2': 'D015742'},
 {'novel': 'Novel',
  'type': 'Comparison',
  'concept_1': 'D003687',
  'concept_2': 'D000661'}]

In [8]:
abstract_offset

[122, 1815]

In [9]:
abstract

'OBJECTIVE: Congenital long QT syndrome (LQTS) with in utero onset of the rhythm disturbances is associated with a poor prognosis. In this study we investigated a newborn patient with fetal bradycardia, 2:1 atrioventricular block and ventricular tachycardia soon after birth. METHODS: Mutational analysis and DNA sequencing were conducted in a newborn. The 2:1 atrioventricular block improved to 1:1 conduction only after intravenous lidocaine infusion or a high dose of mexiletine, which also controlled the ventricular tachycardia. RESULTS: A novel, spontaneous LQTS-3 mutation was identified in the transmembrane segment 6 of domain IV of the Na(v)1.5 cardiac sodium channel, with a G-->A substitution at codon 1763, which changed a valine (GTG) to a methionine (ATG). The proband was heterozygous but the mutation was absent in the parents and the sister. Expression of this mutant channel in tsA201 mammalian cells by site-directed mutagenesis revealed a persistent tetrodotoxin-sensitive but li

In [12]:
def highlight_entities_label(s, start_idx, entities, color='#66CC66'):
    left = f"<span style='background-color: {color}'>"
    right = "</span>"
    final = ""
    last_r = 0
    for entity in entities:
        l, r = entity['offsets'][0]
        if not l >= start_idx:
            continue
        l -= start_idx
        r -= start_idx
        final += s[last_r:l]
        final += left + s[l:r] + right
        last_r = r
    final += s[r:]
    return final

In [13]:
from IPython.display import HTML

label = highlight_entities_label(abstract, abstract_offset[0], entities)
HTML(label)

In [14]:
def highlight_entities_extracted(s, entities, color='#00CCFF'):
    import re
    left = f"<span style='background-color: {color}'>"
    right = "</span>"
    final = s[:]
    for entity in entities:
        entity_text = entity['entity']
        final = re.sub('(?i)'+re.escape(entity_text), lambda m: left+entity_text+right, final)
    return final
predicted = highlight_entities_extracted(abstract, entities_extracted)
HTML(predicted)

TypeError: 'NoneType' object is not iterable

In [25]:
entities_extracted

[{'entity': 'Congenital long QT syndrome (LQTS)',
  'importance': 5,
  'category': 'Disease',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'In utero onset of rhythm disturbances',
  'importance': 3,
  'category': 'BiologicalProcess',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Newborn patient',
  'importance': 4,
  'category': 'Anatomy',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Fetal bradycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': '2:1 atrioventricular block',
  'importance': 4,
  'category': 'Pathway',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Ventricular tachycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Mutational analysis and DNA sequencing',
  'importance': 4,
  'category': 'MolecularFunction',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Lidocaine infusion',
  'importance': 3,
  

In [21]:
import re

In [28]:
confusion_matrix = {"test":[0,0,0]}
categories = []

def match_entity(entity_a, entity_b):
    return entity_a['category'] == entity_b['category'] and\
model.query(entity_a['entity'], top_k=1)[2] == model.query(entity_b['entity'], top_k=1)[2]

def macro_avg_f1_score(ans, gt):
    # for sample_ans, sample_gt in zip(ans, gt):
        # for category in categories:
            # get all entities in the category
    # ans_entities = [entity['entity']
    #                 for entity in sample_ans if entity['category'] == category]
    # ans_entities = [entity['entity'] for entity in ans]
    # # gt_entities = [entity['text']
    # #                for entity in sample_gt if entity['category'] == category]
    # gt_entities = [entity['text'][0] for entity in gt]
    # TP = len(set(ans_entities) & set(gt_entities))
    # FP = len(set(ans_entities) - set(gt_entities))
    # FN = len(set(gt_entities) - set(ans_entities))
    # confusion_matrix["test"][0] += TP
    # confusion_matrix["test"][1] += FP
    # confusion_matrix["test"][2] += FN
    # f1_score = []
    # # for category in categories:
    # TP, FP, FN = confusion_matrix["test"]
    # precision = TP/(TP+FP) if TP+FP else float('nan')
    # recall = TP/(TP+FN) if TP+FN else float('nan')
    # f1_score.append(2*(precision * recall) / (precision + recall))
    # return sum(f1_score) / len(f1_score)

macro_avg_f1_score(entities_extracted, entities)

In [44]:
entities_extracted

[{'entity': 'Congenital long QT syndrome (LQTS)',
  'importance': 5,
  'category': 'Disease',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'In utero onset of rhythm disturbances',
  'importance': 3,
  'category': 'BiologicalProcess',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Newborn patient',
  'importance': 4,
  'category': 'Anatomy',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Fetal bradycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': '2:1 atrioventricular block',
  'importance': 4,
  'category': 'Pathway',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Ventricular tachycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Mutational analysis and DNA sequencing',
  'importance': 4,
  'category': 'MolecularFunction',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Lidocaine infusion',
  'importance': 3,
  

In [10]:
entities[1]

{'text': ['long QT syndrome'],
 'offsets': [[56, 72]],
 'concept_id': 'D008133',
 'semantic_type_id': 'DiseaseOrPhenotypicFeature'}

In [11]:
entities_extracted[0]

{'entity': 'Congenital long QT syndrome (LQTS)',
 'importance': 5,
 'category': 'Disease',
 'pmid': '15485686',
 'type': 'abstract'}

In [10]:
set([j['semantic_type_id'] for i in datas["test"] for j in i['entities']])

{'CellLine',
 'ChemicalEntity',
 'DiseaseOrPhenotypicFeature',
 'GeneOrGeneProduct',
 'OrganismTaxon',
 'SequenceVariant'}

In [3]:
# PROXY
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
    
os.environ['HF_HOME'] = '/autodl-tmp/hf'


from text2knowledge.normalizer import Normalizer

In [4]:
class Arg:
    def __init__(self, model_name_or_path, dictionary_path):
        self.model_name_or_path = model_name_or_path
        self.show_embeddings = False
        self.show_predictions = True
        self.dictionary_path = dictionary_path
        self.use_cuda = True
arg_d = Arg("dmis-lab/biosyn-biobert-ncbi-disease", "/root/text2knowledge/data/dictionary/merged/dict_Disease_20210630.txt")
arg_c = Arg("dmis-lab/biosyn-sapbert-bc5cdr-chemical", "/root/text2knowledge/data/dictionary/dict_ChemicalCompound_20210630.txt")
arg_m = Arg("dmis-lab/biosyn-sapbert-bc5cdr-chemical", "/root/text2knowledge/data/dictionary/yang/dict_Metabolite.txt")
arg_cc = Arg("dmis-lab/biosyn-sapbert-bc5cdr-chemical", "/root/text2knowledge/data/dictionary/yang/dict_CellularComponent.txt")
args_ccc = Arg("dmis-lab/biosyn-biobert-ncbi-disease", "/root/text2knowledge/data/dictionary/yang/dict_CellularComponent.txt")

In [5]:
gene_normalizer = Normalizer(Arg("dmis-lab/biosyn-sapbert-bc2gn", "/root/text2knowledge/data/dictionary/merged/dict_Gene.txt"))

/root/miniconda3/lib/python3.8/site-packages/huggingface_hub/file_download.py:678: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


sparse_encoder.pk:   0%|          | 0.00/68.5k [00:00<?, ?B/s]

/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


sparse_weight.pt:   0%|          | 0.00/824 [00:00<?, ?B/s]

Loaded dictionary from cached file /root/autodl-tmp/tmp/cached_biosyn-sapbert-bc2gn_dict_Gene.pk


In [7]:
s = [0.2, 0.21052631578947364, 0.12903225806451613, 0.5652173913043479, 0.10526315789473684, 0.34146341463414637, 0.19354838709677416, 0, 0.23529411764705882, 0, 0.39999999999999997, 0.5217391304347827, 0.5454545454545455, 0, 0.25454545454545446, 0.35714285714285715, 0.43902439024390244, 0.608695652173913, 0.15384615384615385, 0.38095238095238093, 0.13043478260869565, 0.37500000000000006, 0, 0.08333333333333333, 0.3333333333333333, 0.25, 0.31999999999999995, 0.27027027027027023, 0.689655172413793, 0.02898550724637681, 0.07692307692307691, 0.11764705882352941, 0.17777777777777776, 0.4666666666666667, 0.17391304347826086, 0.039215686274509796, 0, 0, 0.4, 0.3783783783783784, 0.14814814814814814, 0.0909090909090909, 0.15789473684210525, 0, 0.0625, 0, 0, 0.47619047619047616, 0.5, 0.3673469387755102, 0.5945945945945946, 0, 0.5294117647058824, 0, 0, 0.0909090909090909, 0.5, 0.368421052631579, 0.5135135135135135, 0.12121212121212123, 0.15384615384615385, 0.37037037037037035, 0.13157894736842107, 0.08695652173913042, 0.23076923076923078, 0.04444444444444445, 0.711111111111111, 0.20512820512820512, 0.8, 0.12903225806451615, 0, 0.04347826086956522, 0.6666666666666666, 0.8095238095238095, 0.7407407407407407, 0.09523809523809523, 0.5263157894736842, 0, 0.3333333333333333, 0.08695652173913045, 0.05405405405405406, 0.05263157894736842, 0.5, 0.15384615384615385, 0.4838709677419354, 0.05]
sum(s) / len(s)

0.2550491213744335

In [8]:
gene_normalizer.normalize('Na(v)1.5 cardiac sodium channel')

/root/miniconda3/lib/python3.8/site-packages/biosyn-0.1-py3.8.egg/biosyn/biosyn.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


{'mention': 'na v 1 5 cardiac sodium channel',
 'predictions': [{'name': 'voltage-gated sodium channel subunit alpha Nav1.5',
   'id': 'EntrezGene:6331'},
  {'name': 'sodium channel, voltage-gated, type V, alpha',
   'id': 'ENTREZ:20271'},
  {'name': 'SCN5A', 'id': 'EntrezGene:6331|11311'},
  {'name': 'voltage-gated sodium channel subunit alpha Nav1.1',
   'id': 'EntrezGene:6323'},
  {'name': 'voltage-gated sodium channel subunit alpha Nav1.3',
   'id': 'EntrezGene:6328'}]}

In [3]:
disease_normalizer = Normalizer(Arg("dmis-lab/biosyn-biobert-ncbi-disease", "/root/text2knowledge/data/dictionary/merged/dict_Disease.txt"))

/root/miniconda3/lib/python3.8/site-packages/huggingface_hub/file_download.py:678: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainabi

Loaded dictionary from cached file /root/autodl-tmp/tmp/cached_biosyn-biobert-ncbi-disease_dict_Disease.pk


In [4]:
disease_normalizer.normalize('death')

/root/miniconda3/lib/python3.8/site-packages/biosyn-0.1-py3.8.egg/biosyn/biosyn.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


{'mention': 'death',
 'predictions': [{'name': 'Death, Sudden|Sudden Death', 'id': 'MESH:D003645'},
  {'name': 'exhaustion', 'id': 'SYMP:0000186'},
  {'name': 'Familial Sudden Death', 'id': 'MESH:C566172'},
  {'name': 'complete', 'id': 'MONDO:0700063'},
  {'name': 'Cachexia', 'id': 'MESH:D002100'}]}

In [5]:
chemical_normalizer3 = Normalizer(arg_cc)
chemical_normalizer4 = Normalizer(args_ccc)

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sparse_encoder.pk:   0%|          | 0.00/66.9k [00:00<?, ?B/s]

/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


sparse_weight.pt:   0%|          | 0.00/392 [00:00<?, ?B/s]

In [10]:
chemical_normalizer.normalize("Cyclin-dependent kinase")

/root/miniconda3/lib/python3.8/site-packages/biosyn-0.1-py3.8.egg/biosyn/biosyn.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Loaded dictionary from cached file /root/autodl-tmp/tmp/cached_biosyn-sapbert-bc5cdr-chemical_dict_ChemicalCompound_20210630.pk


{'mention': 'cyclin dependent kinase',
 'predictions': [{'name': 'EC 2.7.3.2 (creatine kinase) inhibitor|EC 2.7.3.2 (creatine kinase) inhibitors|CK inhibitors|creatine kinase inhibitor|creatine kinase (EC 2.7.3.2) inhibitor|creatine kinase (EC 2.7.3.2) inhibitors|CPK inhibitor|creatine phosphokinase inhibitors|creatine kinase inhibitors|CPK inhibitors|CK inhibitor|creatine phosphokinase inhibitor|cpkinhibitor|cpkinhibitors|creatinephosphokinaseinhibitor|ckinhibitors|creatinekinaseinhibitors|ckinhibitor|creatinekinaseinhibitor|creatinephosphokinaseinhibitors',
   'id': 'CHEBI:73354'},
  {'name': 'cladieunicellin K', 'id': 'MESH:C000588686'},
  {'name': 'cyclic ketone|Cyclic ketone|cyclicketones|cyclicketone',
   'id': 'CHEBI:3992'},
  {'name': "2',3'-dideoxycarbocyclic thymidine", 'id': 'MESH:C585686'},
  {'name': 'CpG-KSK13|cpgksk13', 'id': 'MESH:C544079'}]}

In [6]:
chemical_normalizer3.normalize("Cyclin-dependent kinase")

Loaded dictionary from cached file /root/autodl-tmp/tmp/cached_biosyn-sapbert-bc5cdr-chemical_dict_CellularComponent.pk


/root/miniconda3/lib/python3.8/site-packages/biosyn-0.1-py3.8.egg/biosyn/biosyn.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'mention': 'cyclin dependent kinase',
 'predictions': [{'name': 'cytoplasmic cyclin-dependent protein kinase holoenzyme complex|CDK holoenzyme',
   'id': 'GO:0000308'},
  {'name': 'cyclin-dependent protein kinase holoenzyme complex|CDK holoenzyme',
   'id': 'GO:0000307'},
  {'name': 'nuclear cyclin-dependent protein kinase holoenzyme complex|CDK holoenzyme',
   'id': 'GO:0019908'},
  {'name': 'cyclin E2-CDK2 complex', 'id': 'GO:0097135'},
  {'name': 'cyclin D2-CDK6 complex', 'id': 'GO:0097132'}]}

In [7]:
chemical_normalizer4.normalize("Cyclin-dependent kinase")

/root/miniconda3/lib/python3.8/site-packages/biosyn-0.1-py3.8.egg/biosyn/biosyn.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
embedding dictionary: 100%|██████████| 5/5 [00:00<00:00,  5.93it/s]

Saving dictionary into cached file /root/autodl-tmp/tmp/cached_biosyn-biobert-ncbi-disease_dict_CellularComponent.pk


{'mention': 'cyclin dependent kinase',
 'predictions': [{'name': 'cytoplasmic cyclin-dependent protein kinase holoenzyme complex|CDK holoenzyme',
   'id': 'GO:0000308'},
  {'name': 'cyclin E1-CDK2 complex', 'id': 'GO:0097134'},
  {'name': 'cyclin-dependent protein kinase holoenzyme complex|CDK holoenzyme',
   'id': 'GO:0000307'},
  {'name': 'cyclin D2-CDK6 complex', 'id': 'GO:0097132'},
  {'name': 'cyclin E2-CDK2 complex', 'id': 'GO:0097135'}]}

In [7]:
chemical_normalizer3.normalize("Phosphatidylinositol")

Loaded dictionary from cached file /root/autodl-tmp/tmp/cached_biosyn-sapbert-bc5cdr-chemical_dict_CellularComponent.pk


{'mention': 'phosphatidylinositol',
 'predictions': [{'name': 'piP-body', 'id': 'GO:0071547'},
  {'name': 'polytene chromosome interband', 'id': 'GO:0005705'},
  {'name': 'pterinosome', 'id': 'GO:0043700'},
  {'name': 'Pi Mi complex', 'id': 'GO:0062071'},
  {'name': 'plasma membrane|juxtamembrane|cellular membrane|cell membrane|cytoplasmic membrane|plasmalemma|integral component of plasma membrane|bacterial inner membrane|inner endospore membrane|plasma membrane lipid bilayer|integral to plasma membrane',
   'id': 'GO:0005886'}]}

In [5]:
disease_normalizer.normalize("ovarian endometrioid cystadenofibroma")

Loaded dictionary from cached file ./tmp/cached_biosyn-biobert-ncbi-disease_dict_Disease_20210630.pk


{'mention': 'ovarian endometrioid cystadenofibroma',
 'predictions': [{'name': 'Ovarian Fibromata', 'id': 'MESH:C562391'},
  {'name': 'Ovarian gynandroblastoma', 'id': 'MESH:C538459'},
  {'name': 'Teratoma, Ovarian|Dermoid Cyst, Ovarian', 'id': 'MESH:C562731'},
  {'name': 'Androblastoma of ovary|Sertoli-leydig cell tumor of the ovary',
   'id': 'MESH:C537588'},
  {'name': 'Neonatal ovarian cyst|Fetal ovarian cyst', 'id': 'MESH:C536396'}]}

In [6]:
disease_normalizer.normalize("hypochondriacal disorder")

Loaded dictionary from cached file ./tmp/cached_biosyn-biobert-ncbi-disease_dict_Disease_20210630.pk


{'mention': 'hypochondriacal disorder',
 'predictions': [{'name': 'Hypochondriacal neurosis|Hypochondria|HYPOCHONDRIASIS|Hypochondriacal Neuroses|hypochondriacal neurosis|Hypochondriacal disorder|Hypochondriasis|Hypochondriacal Neurosis|Neurosis, Hypochondriacal',
   'id': 'MESH:D006998'},
  {'name': 'Premenstrual Dysphoric Disorder|Disorder, Premenstrual Dysphoric|Dysphoric Disorder, Premenstrual|Premenstrual Dysphoric Syndrome|Syndrome, Premenstrual Dysphoric',
   'id': 'MESH:D065446'},
  {'name': 'Schizophrenia and Disorders with Psychotic Features',
   'id': 'MESH:D019967'},
  {'name': 'Trauma and Stressor Related Disorders|', 'id': 'MESH:D000068099'},
  {'name': 'Ameloonychohypohidrotic syndrome|Hypocalcified-hypoplastic enamel, onycholysis with subungual hyperkeratosis, and hypohidrosis',
   'id': 'MESH:C538245'}]}

In [21]:
import pprint
for disease_entity in disease_entities_extracted:
    result = disease_normalizer.normalize(disease_entity["entity"])
    pprint.pp(result)

Loaded dictionary from cached file ./tmp/cached_biosyn-biobert-ncbi-disease_dict_Disease_20210630.pk
{'mention': 'nephrogenic diabetes insipidus ndi',
 'predictions': [{'name': 'diabetes insipidus, nephrogenic, x-linked|congenital nephrogenic diabetes insipidus|diabetes insipidus, nephrogenic, type ii|vasopressin-resistant diabetes insipidus|nephrogenic diabetes insipidus, type is|diabetes insipidus, nephrogenic, autosomal|acquired nephrogenic diabetes insipiduses|congenital nephrogenic diabetes insipiduses|acquired nephrogenic diabetes insipidus|diabetes insipidus renalis|adh-resistant diabetes insipidus|nephrogenic diabetes insipidus|adh-resistant diabetes insipiduses|diabetes insipidus, nephrogenics|diabetes insipidus, nephrogenic, type 1|diabetes insipidus, nephrogenic|diabetes insipidus, nephrogenic, type i|nephrogenic diabetes insipidus, type iis|nephrogenic diabetes insipidus, type i|nephrogenic diabetes insipidus, type ii|vasopressin resistant diabetes insipidus|Familial nephro

In [6]:
confusion_matrix = {"test":[0,0,0]}
categories = []

def match_entity(entity_a, entity_b):
    return entity_a['category'] == entity_b['category'] and\
model.query(entity_a['entity'], top_k=1)[2] == model.query(entity_b['entity'], top_k=1)[2]

def macro_avg_f1_score(ans, gt):
    pass
    # for sample_ans, sample_gt in zip(ans, gt):
        # for category in categories:
            # get all entities in the category
    # ans_entities = [entity['entity']
    #                 for entity in sample_ans if entity['category'] == category]
    # ans_entities = [entity['entity'] for entity in ans]
    # # gt_entities = [entity['text']
    # #                for entity in sample_gt if entity['category'] == category]
    # gt_entities = [entity['text'][0] for entity in gt]
    # TP = len(set(ans_entities) & set(gt_entities))
    # FP = len(set(ans_entities) - set(gt_entities))
    # FN = len(set(gt_entities) - set(ans_entities))
    # confusion_matrix["test"][0] += TP
    # confusion_matrix["test"][1] += FP
    # confusion_matrix["test"][2] += FN
    # f1_score = []
    # # for category in categories:
    # TP, FP, FN = confusion_matrix["test"]
    # precision = TP/(TP+FP) if TP+FP else float('nan')
    # recall = TP/(TP+FN) if TP+FN else float('nan')
    # f1_score.append(2*(precision * recall) / (precision + recall))
    # return sum(f1_score) / len(f1_score)

macro_avg_f1_score(entities_extracted, entities)

NameError: name 'entities_extracted' is not defined

In [1]:
1

1